In [ ]:
import watertap as wt
import pandas as pd
from pyomo.environ import value, Block
from idaes.core import FlowsheetBlock
import numpy as np
from case_study_trains import *
import case_study_trains
import pyomo.environ as env
from pyomo.environ import Constraint


In [ ]:
df = pd.read_csv("data/baseline_cases_runs.csv")
reference = "nawi"

In [ ]:
for source_water in ["industrial"]: #df.source_water_category.unique():
    for case_study in ["upw"]: #df[df.source_water_category == source_water].case_study.unique():
        h1 = df[df.case_study == case_study]

        if case_study in ["upw"]:
            scenarios = ["zld_a", "zld_b", "zld_c", "baseline"]
        else:        
            scenarios = h1.scenario.unique()

        for scenario in scenarios:

            print("------------------------------------------------------")
            print("------------------------------------------------------")
            print("-----------", case_study, scenario, "-----------------")
            print("------------------------------------------------------")
            print("------------------------------------------------------")

            desired_recovery = df[((df.case_study == case_study) & (df.scenario == scenario))].max_recovery_rate.max()

            source_water_category = df[((df.case_study == case_study) & (df.scenario == scenario))].source_water_category.max()

            skip_small_base = df[((df.case_study == case_study) & (df.scenario == scenario))].skip_small_base.max()
            skip_small_sens = df[((df.case_study == case_study) & (df.scenario == scenario))].skip_small_sens.max()
            ro_bounds = df[((df.case_study == case_study) & (df.scenario == scenario))].ro_bounds.max()

            m = wt.watertap_setup(dynamic=False, case_study = case_study, reference = reference, 
                                  scenario = scenario, source_scenario = "baseline")

            m = wt.case_study_trains.get_case_study(m=m)

            for key in m.fs.pfd_dict.keys():
                if m.fs.pfd_dict[key]["Unit"] == "reverse_osmosis":
                    getattr(m.fs, key).a.fix(4.2)
                    getattr(m.fs, key).b.fix(0.35)

            if "1p5" in scenario:
                m.fs.irrigation_and_drainage.flow_vol_in.fix(0.0657)

            # run and and return baseline with 0 degrees of freedom
            m = wt.run_water_tap_ro(m, source_water_category = source_water_category, 
                                 desired_recovery = desired_recovery, skip_small = skip_small_base, 
                                 ro_bounds = ro_bounds, source_scenario = "baseline", scenario_name = scenario)

            if scenario == "baseline":
                wt.sensitivity_runs(m = m, save_results = True, scenario = scenario, 
                                    case_study = case_study, skip_small_sens = skip_small_sens)

                if source_water == "seawater":
                    m = wt.get_fixed_onm_reduction(m)
                    wt.run_water_tap(m=m, objective=True, skip_small=skip_small_base, print_model_results="summary")
                    df_no2 = wt.get_results_table(m=m, case_study=m.fs.train["case_study"], 
                                              scenario="50%_fixed_onm_reduction")

                if case_study == "san_luis":
                    n = 1
                    for flow_scenario in [0.0657, 0.1314, 0.2191]:
                        m.fs.irrigation_and_drainage.flow_vol_in.fix(flow_scenario)
                        m.fs.reverse_osmosis.feed.pressure.unfix()
                        m.fs.reverse_osmosis.membrane_area.unfix()

                        print("adjusted baseline flow -->", flow_scenario)
                        wt.run_water_tap(m=m, objective=True, skip_small=skip_small_base, print_model_results="summary")
                        if n == 1: s_name = m.fs.train["case_study"] + "1p5_mgd"
                        if n == 2: s_name = m.fs.train["case_study"] + "3_mgd"
                        if n == 3: s_name = m.fs.train["case_study"] + "5_mgd"
                        df_no2 = wt.get_results_table(m=m, case_study=m.fs.train["case_study"], scenario=s_name)                
                        n = n + 1

            dwi_list = ["emwd", "big_spring"]   
            if m.fs.train["case_study"] in dwi_list:
                for key in m.fs.pfd_dict.keys():
                    if m.fs.pfd_dict[key]["Unit"] == "deep_well_injection":
                        wt.sensitivity_runs(m = m, save_results = True, scenario = scenario, 
                                            case_study = case_study, skip_small_sens = skip_small_sens)

            if scenario in ["edr_ph_ro", "ro_and_mf"]:
                print("goes in")
                wt.sensitivity_runs(m = m, save_results = True, scenario = scenario, 
                                    case_study = case_study, skip_small_sens = skip_small_sens)                

            if source_water == "power":
                for key in m.fs.pfd_dict.keys():
                    if m.fs.pfd_dict[key]["Unit"] == "evaporation_pond":
                        wt.sensitivity_runs(m = m, save_results = True, scenario = scenario, 
                                            case_study = case_study, skip_small_sens = skip_small_sens)  



In [ ]:
wt.display.show_train2(model_name=m)

In [ ]:
#wt.run_water_tap(m=m, objective=False, print_model_results="summary", skip_small=True)

In [ ]:
import pyomo.util.infeasible as infeas
print(infeas.log_infeasible_bounds(m))
print(infeas.log_infeasible_constraints(m))